In [25]:
%reset

In [26]:
import numpy as np
import sympy as sp
import math
import scipy.integrate as si
import scipy.optimize as so

In [27]:
# PARAMETERS

concentration = 10 # concentration of pre-encounter minihalo
Efrac_min = 1e-4 # minimum value of E_frac to be considered
Efrac_max = 1e3  # maximum value of E_frac to be considered
num_Efracs = 41 # total number of E_frac values to be considered, logarithmically spaced between 'Efrac_min' and 'Efrac_max'
save_to_disk = False # If True, saves the E_frac array and survival fractions array to disk. Stored as '.npy' files. Set to 'False' if you don't want to save arrays to disk.

In [28]:
def second_derivative_parametric_input(x, y, t):
    # Input should be 'x' in terms 't', and 'y' in terms of 't'
    # Output is second derivative of 'y' with respect to 'x' ('d2y/dx2') as a function of 't'
    dy_dx = sp.Derivative(y, t) / sp.Derivative(x, t)
    d2y_dx2 =  sp.Derivative(dy_dx, t) / sp.Derivative(x, t)
    d2y_dx2_in_terms_of_t = sp.nsimplify(d2y_dx2.doit())

    return d2y_dx2_in_terms_of_t
    

In [29]:
c, f_NFW, x_prime = sp.symbols("c f_NFW x_prime", positive=True)
crho = 1 / (c*x_prime) / (1+c*x_prime)**2  # dimensionless density of NFW profile. 'chro = rho / rho_s', where 'rho' is the density profile and 'rho_s' is the scale radius.
psi_2 = 1/f_NFW * ( sp.log(1+c*x_prime)/x_prime - c/(1+c*x_prime) ) # normalized relative potential

d2crho_dpsi2 = second_derivative_parametric_input(psi_2, crho, x_prime)  # Second derivative of 'chro' with respect to 'psi_2' as a functin of 'x_prime'

In [30]:
c_val = concentration
f_NFW_val = math.log(1+c_val)-c_val/(1+c_val)

alpha_squared = 3/c_val**2 + 1/(2*f_NFW_val)*(c_val-3)/(c_val+1)
beta = (c_val**3 - 2*c_val*(1+c_val)*f_NFW_val) / (2*(1+c_val)**2 * f_NFW_val**2) # this is actually gamma in our article

d2crho_dpsi2_c_and_fNFW_substituted = d2crho_dpsi2.subs({c:c_val, f_NFW:f_NFW_val}) # substitute the numerical values of 'concentration' and 'f_NFW' in 'd2crho_dpsi2'

d2crho_dpsi2_c_and_fNFW_substituted_lambdified = sp.lambdify(x_prime, d2crho_dpsi2_c_and_fNFW_substituted) # Lambdify 'd2crho_dpsi2_c_and_fNFW_substituted' so that you can evaluate it by passing a numerical value of 
                                                                                                           # 'x_prime' as input

# Compute the normalized crossover radius x*
def get_x_c():
    eqn = lambda x_c: beta / alpha_squared * E_frac * x_c**2 - ( 1/f_NFW_val * ( np.log(1+c_val*x_c) / x_c  -  c_val / (1+c_val*x_c) ) )
    sol = so.fsolve(eqn, 1)[0]
    
    return sol

# Computes normalized radius 'x' given a value of normalized relative potential 'psi_B', using equation (25) in our article 
def get_x_prime(psi_prime):
    eqn = lambda x_Prime: psi_prime  -  1/f_NFW_val * ( np.log(1+c_val*x_Prime)/x_Prime - c_val/(1+c_val*x_Prime) )
    #sol = so.fsolve(eqn, 1)[0]
    sol = so.root_scalar(eqn, bracket=(x_star, 1e20)).root
    #print(sol)
    
    return sol

# Compute the integrand according to equation (23) in our article
def integrand_1(psi_prime, eps, x):
    
    psi_1 = 1/f_NFW_val * ( math.log(1+c_val*x)/x - c_val/(1+c_val*x_star) ) # This is 'psi_A' from equation (24) in our article
    x_prime = get_x_prime(psi_prime)

    return (
        x**2 * 1/(math.sqrt(8)*math.pi**2) * 1/np.sqrt(eps - psi_prime) * 
        d2crho_dpsi2_c_and_fNFW_substituted_lambdified(x_prime).real * np.sqrt(2*(psi_1 - eps))
    )

def range_x_1():
    return [0, min(x_star, 1)]

def range_eps_1(x):
    return [0, beta / alpha_squared * E_frac * x**2]

def range_psi_prime_1(eps, x):
    return [0, eps]



# Compute the integrand according to equation (21) in our article
def integrand_2(x):
    return x / (1 + c_val*x)**2

def range_x_2():
    return [0, min(x_star, 1)]



    
E_fracs = np.geomspace(Efrac_min, Efrac_max, num_Efracs, endpoint=True) # Creates a logarithmically spaced array of E_frac values. Adjust as necessary
survival_fractions = np.zeros(np.size(E_fracs)) # Create a zeros array the same size as the 'E_fracs' array. This will eventually contain the values of survival fractions corresponding to E_frac values.

for i in range(np.size(E_fracs)):

    E_frac = E_fracs[i]

    x_star = get_x_c() # compute the normalized crossover radius x*

    # Compute mass of 0 < x < x_star region assuming no mass loss in this region - computes the survival fraction equivalent
    mass_fraction = c_val**2/f_NFW_val * si.nquad(integrand_2, [range_x_2])[0]

    # Compute partial mass loss in 0 < x < x_star region - computes the survival fraction equivalent
    partial_mass_loss_fraction = 4*math.pi*c_val**3/f_NFW_val * si.nquad(integrand_1, [range_psi_prime_1, range_eps_1, range_x_1])[0]

    survival_fraction = mass_fraction - partial_mass_loss_fraction # equation (20) in our article
    
    survival_fractions[i] = survival_fraction

    print(f"Iteration #{i+1}", end="\r", flush=True)


In [31]:
# Save files to disk
if save_to_disk:
    np.save("E_fracs", E_fracs)
    np.save(f"survival_fractions_c_{c_val}_sequential_stripping", survival_fractions)